In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pwd

/content


In [ ]:
%cd /content/drive/Shareddrives/CSE573/preprocessed
%ls

/content/drive/Shareddrives/CSE573/preprocessed
AaplAmzn_NewsDateDirection.csv
Aapl_NewsPriceHistDirection.csv
Aapl_SentimentAggMeanPriceHistDirection.csv
Aapl_SentimentAggSumPriceHistDirection.csv
aapl_text_direction_date.csv
Amzn_NewsPriceHistDirection.csv
Amzn_SentimentAggMeanPriceHistDirection.csv
Amzn_SentimentAggSumPriceHistDirection.csv
amzn_title_processedText_direction.csv
amzn_title_processedText_direction_date.csv
amzn_title_text_label.csv
lstm_amzn_fulltext/
text_kfold_aapl.csv


In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf
from collections import Counter
from keras.preprocessing.text import Tokenizer
from keras.utils import pad_sequences

import requests
import io

In [ ]:
df_amzn = pd.read_csv('amzn_title_processedText_direction_date.csv')


In [ ]:
df_amzn.head()

,title,text,direction,text_without_stopwords,processedText,prediction_date
0,6 things to know before the opening bell,click chart for more in-depth data. 1. big tec...,-1,click chart in-depth data 1. big tech senate b...,click chart in-depth data 1. big tech senate b...,2018-09-05
1,"Netflix And The Competition - Netflix, Inc. (N...",investment thesis netflix ( nflx ) is a remark...,-1,investment thesis netflix nflx remarkable serv...,investment thesis netflix nflx remarkable serv...,2018-09-05
2,Pandora Stock Gets a Boost — Buy Now,mannkind surges more than 50% >>> read more\nb...,-1,mannkind surges 50 read breaking news sponsore...,surges 50 read breaking news sponsored home st...,2018-09-05
3,MARKET SNAPSHOT: Tech Tumble Weighs On Stocks;...,"by mark decambre, marketwatch , ryan vlastelic...",-1,mark decambre marketwatch ryan vlastelica tech...,mark decambre marketwatch ryan vlastelica tech...,2018-09-05
4,"New Month, Same Old Story: Trade Fears Persist...","it may be a new month, but the same trade conc...",-1,may new month trade concerns haunted market ye...,may new month trade concerns market year remai...,2018-09-05


In [ ]:
from tensorflow.keras.layers import TextVectorization

In [ ]:
#title_amzn=df_amzn['title'].tolist()
text_amzn=df_amzn['processedText'].to_numpy()
label_amzn=df_amzn['direction'].to_numpy()


#print(title_amzn[0])
print(text_amzn[0])
print(label_amzn[0])

click chart in-depth data 1. big tech senate big tech executives coming capitol hill wednesday testify election meddling facebook fb coo sheryl sandberg jack dorsey ceo twitter twtr appear senate intelligence committee discuss actions companies taken foreign influence campaigns targeting 2018 midterm elections committee requested high-profile executive google owns youtube also appear testify google googl parent company alphabet however declined send one top people 2. emerging markets fears emerging markets currencies tumbled dollar wednesday mexican south indian among biggest losers rising us interest rates political global trade war recent months exposed multiple emerging markets turkey argentina hardest hit investors worried adding fears markit index measures emerging markets manufacturing fell fresh low august watch cnnmoney now today p.m. et 3. global stocks overview us stock futures pointing lower european markets opened down exchanges london frankfurt paris dropping roughly 0.5 a

In [ ]:
class_label_amzn = (label_amzn + 1) / 2
#class_label_amzn = pd.get_dummies(label_amzn).values

In [ ]:
encoder = tf.keras.layers.TextVectorization(standardize=None)
encoder.adapt(text_amzn)

In [ ]:

print(encoder(text_amzn[:3]).numpy())

[[ 458  228 3189 ...    0    0    0]
 [  98 2656   58 ...    4  722  181]
 [7351  373  211 ...    0    0    0]]


In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(text_amzn, class_label_amzn, test_size=0.2, random_state=56)

In [ ]:
X_train.shape

(15262,)

In [ ]:
Y_train.shape

(15262,)

In [ ]:
from tensorflow.keras.layers import Bidirectional, LSTM, Dense, Embedding
from keras.optimizers import Adam, SGD
from tensorflow.keras.metrics import Recall, Precision

In [ ]:
layers = [
    encoder,
    tf.keras.layers.Embedding(
        input_dim=len(encoder.get_vocabulary()),
        output_dim=128,
        mask_zero=True),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(128)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1)
]
model = tf.keras.models.Sequential(layers)
model.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              optimizer=tf.keras.optimizers.Adam(1e-4),
              metrics=['accuracy'])
model.fit(X_train[:1000], Y_train[:1000], epochs=10, batch_size=64, verbose=1,validation_split=0.2)

Epoch 1/10
13/13 [==============================] - 216s 16s/step - loss: 0.6929 - accuracy: 0.4800 - val_loss: 0.6931 - val_accuracy: 0.5100
Epoch 2/10
13/13 [==============================] - 182s 15s/step - loss: 0.6912 - accuracy: 0.4800 - val_loss: 0.6931 - val_accuracy: 0.5100
Epoch 3/10
13/13 [==============================] - 211s 16s/step - loss: 0.6897 - accuracy: 0.4800 - val_loss: 0.6931 - val_accuracy: 0.5100
Epoch 4/10
13/13 [==============================] - 194s 14s/step - loss: 0.6878 - accuracy: 0.4800 - val_loss: 0.6931 - val_accuracy: 0.5100
Epoch 5/10
13/13 [==============================] - 208s 17s/step - loss: 0.6851 - accuracy: 0.4800 - val_loss: 0.6932 - val_accuracy: 0.5100
Epoch 6/10
13/13 [==============================] - 237s 19s/step - loss: 0.6806 - accuracy: 0.4800 - val_loss: 0.6934 - val_accuracy: 0.5100
Epoch 7/10
13/13 [==============================] - 241s 19s/step - loss: 0.6722 - accuracy: 0.4800 - val_loss: 0.6938 - val_accuracy: 0.5100
Epoch 

In [ ]:
model.fit(X_train[:1000], Y_train[:1000], epochs=10, batch_size=64, verbose=1,validation_split=0.2)

Epoch 1/10
13/13 [==============================] - 221s 18s/step - loss: 0.4505 - accuracy: 0.8537 - val_loss: 0.7027 - val_accuracy: 0.5900
Epoch 2/10
13/13 [==============================] - 201s 14s/step - loss: 0.3588 - accuracy: 0.8800 - val_loss: 0.7416 - val_accuracy: 0.5750
Epoch 3/10
13/13 [==============================] - 192s 15s/step - loss: 0.3010 - accuracy: 0.8900 - val_loss: 0.7480 - val_accuracy: 0.5450
Epoch 4/10
13/13 [==============================] - 210s 17s/step - loss: 0.2216 - accuracy: 0.9375 - val_loss: 0.8914 - val_accuracy: 0.5750
Epoch 5/10
13/13 [==============================] - 224s 18s/step - loss: 0.1355 - accuracy: 0.9650 - val_loss: 0.9092 - val_accuracy: 0.5650
Epoch 6/10
13/13 [==============================] - 219s 16s/step - loss: 0.0825 - accuracy: 0.9775 - val_loss: 1.0115 - val_accuracy: 0.5650
Epoch 7/10
13/13 [==============================] - 221s 17s/step - loss: 0.0562 - accuracy: 0.9875 - val_loss: 1.0946 - val_accuracy: 0.5800
Epoch 

In [ ]:
eva = model.evaluate(X_test, Y_test, verbose=2)

120/120 - 260s - loss: 1.2942 - accuracy: 0.5453 - 260s/epoch - 2s/step
